In [ ]:
from atl_module import GebcoUpscaler
import pandas as pd
from sklearn.metrics import mean_squared_error

In [ ]:
%cd ..

In [ ]:
site = GebcoUpscaler("oahu",
                     '../data/test_sites/oahu/in-situ-DEM/truth.vrt'
                    )

In [ ]:
# site.download_ATL03()

In [ ]:
# site.recalc_tracklines_gdf()

In [ ]:
# site.subset_gebco(hres=50)

In [ ]:
# from itertools import product
# # setup result dictionary
# result = {}
# # make an empty list for each result
# result['window']=[]
# result['perc_hconf']=[]
# result['min_kde']=[]
# result['rmse']=[]
# result['MAE']=[]

# # loop over the inner product of the 
# for window,perc_h,kde_min in product([100,200],[20,40,60],[0.0,0.1,0.2]):
#     result['window'].append(window)
#     result['perc_hconf'].append(perc_h)
#     result['min_kde'].append(kde_min)
#     site.find_bathy_from_icesat(
#     window=window,
#     threshold_val=0.0,
#     req_perc_hconf=perc_h,
#     window_meters=None,
#     min_photons=None,
#     min_kde=kde_min,
#     save_result=False
#     )
#     site.lidar_rmse()
#     result['rmse'].append(site.rmse_icesat)
#     result['MAE'].append(site.mae_icesat)
#     print(result)

# pd.DataFrame(result).to_csv(f'../data/test_sites/{site.site}/error_stats.csv')

In [ ]:
site.find_bathy_from_icesat(
    window=100,
    threshold_val=0.0,
    req_perc_hconf=40,
    window_meters=None,
    min_photons=None,
    min_kde=0.1
)
site.lidar_error()

In [ ]:
site.bathy_pts_gdf = site.bathy_pts_gdf.assign(z_kde = site.bathy_pts_gdf.z_kde - site.bathy_pts_gdf.dac_corr  + site.bathy_pts_gdf.tide_ocean_corr)

In [ ]:
%%time
site.kriging(
    npts=3000,
    samplemethod='dart',
    # variogram_parameters={'range':33000,'nugget':0.7,'sill':23},

)

In [ ]:
site.kalman_update(1.5)
site.raster_rmse()

In [ ]:
date_indexed_tracklines  = site.tracklines.assign(date = pd.to_datetime(site.tracklines.date).dt.date).reset_index().set_index(['date','beam'])
df = site.bathy_pts_gdf.eval('error=z_kde-true_elevation').eval('error_abs=abs(z_kde-true_elevation)').groupby([pd.to_datetime(site.bathy_pts_gdf.rename(columns={'delta_time':'date'}).date).dt.date,'beam']).mean().sort_values('error',ascending=False)
# 
df = df.merge(date_indexed_tracklines,how='left',left_index=True,right_index=True)


In [ ]:
site.bathy_pts_gdf.eval('error=z_kde-true_elevation').plot.scatter(x='error',y='sea_level_interp')

In [ ]:
site.bathy_pts_gdf.eval('error=(z_kde-true_elevation)').plot.scatter(x='error',y='kde_val')

In [ ]:
ax = site.bathy_pts_gdf.plot.scatter(x='true_elevation',y='z_kde',xlabel='True Elevation [m +MSL]',ylabel='Calculated Elevation [m +MSL]')

ax.plot((-17.5,0),(-17.5,0),c='red')
ax.get_figure().savefig(f'../document/figures/{site.site}_estimated_vs_truth.jpg',facecolor='white',bbox_inches='tight',dpi=800)

In [ ]:
df.plot.scatter(y='error',x='oc_sig_conf')

In [ ]:
df.plot.scatter(y='error',x='oc_hconf_perc')

In [ ]:
df.plot.scatter(y='error',x='secchi_depth')

In [ ]:
df.plot.scatter(y='error',x='tide_ocean_corr')

In [ ]:
df.plot.scatter(y='error',x='dac_corr')

In [ ]:
df.plot.scatter(y='error',x='kde_val')

In [ ]:
df.plot.scatter(y='error',x='sea_level_interp')

In [ ]:
site.bathy_pts_gdf.eval('error=z_kde-true_elevation').corr()['error'].sort_values()

In [ ]:
df.corr()['error'].sort_values()

In [ ]:
site.bathy_pts_gdf[site.bathy_pts_gdf.Z_geoid>-50].dropna().eval('error=(z_kde-true_elevation)**2').error.mean()**(0.5)

In [ ]:
site.bathy_pts_gdf.eval('error=z_kde-true_elevation').error.mean()

In [ ]:
dac_tide = site.bathy_pts_gdf.z_kde  - site.bathy_pts_gdf.dac_corr  + site.bathy_pts_gdf.tide_ocean_corr
site.bathy_pts_gdf.assign(dac_tide=dac_tide).eval('error=(dac_tide-true_elevation)**2').error.mean()**(0.5)

In [ ]:
site.bathy_pts_gdf.assign(dac_tide=dac_tide).eval('error=dac_tide-true_elevation').error.mean()

In [ ]:
site.bathy_pts_gdf[site.bathy_pts_gdf.Z_geoid>-50].sea_level_interp

In [ ]:
df.secchi_depth.mean()

## 